In [1]:
import codecs
from bs4 import BeautifulSoup
import csv

## Open HTML DOM file and intialize variables

In [2]:
input_file_path='Wiki Application.html'
output_file_name='Wiki_UI_Components.csv'
f=codecs.open(file_path, 'r', 'utf-8')
dic = dict()
ls=[]
itt = 0
rows=[]

## Function to generate Xpath of elements

In [3]:
def xpath_soup(element):
    # type: (typing.Union[bs4.element.Tag, bs4.element.NavigableString]) -> str
    """
    Generate xpath from BeautifulSoup4 element.
    """
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:  # type: bs4.element.Tag
        siblings = parent.find_all(child.name, recursive=False)
        components.append(
            child.name if 1 == len(siblings) else '%s[%d]' % (
                child.name,
                next(i for i, s in enumerate(siblings, 1) if s is child)
                )
            )
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

## Recursive Function to parse HTML elements

In [4]:
def list_tree_names(node):
    attributes=["href","class","role","target","src", "alt", "title", "type","scope"]
    empty_tags=["area","br","hr","input","meta","base","col","img","link","param"]
    
    #global xpath
    global itt    
    xpath=(xpath_soup(node))
    #print(node.contents)
    for child in node.contents:
        try:
            dic.update({child.name + "/" + str(itt): child.attrs})
            ls.append(child.name)
            itt += 1
            if child.name in empty_tags:
                #print("Tag:"+child.name)
                #print("Attrs:"+str(child.attrs))
                #print(xpath)
                row=[str(child.name),str("")]
                for atri in attributes:
                    try:
                        row.append(str(child.attrs[atri]))
                    except:
                        row.append("") 
                row.append(xpath)
                rows.append(row)
            list_tree_names(node=child)
        except:
            
            if len(child)>1:
                #print(len(child))
                #print("Tag:"+child.parent.name)
                #print("Text:"+child)
                #print("Attrs:"+str(child.parent.attrs))
                #print(xpath)
                row=[str(child.parent.name),str(child)]
                for atri in attributes:
                    try:
                        row.append(str(child.parent.attrs[atri]))
                    except:
                        row.append("") 
                row.append(xpath)
                rows.append(row)
            dic.update({"text" + "/" + str(itt): child})
            ls.append(child)
            itt += 1
    return rows

## Html file using beautiful soup library

In [5]:
soup = BeautifulSoup(f.read(), "html.parser")

In [ ]:
list_tree_names(soup.find('body'))

## Export all extracted elements and its attributes, xpath in csv file.

In [7]:
with open(output_file_name, 'w', newline='',encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["tag_name", "text",'href','class','role','target','src', 'alt', 'title', 'type','scope',"xpath"])
        writer.writerows(rows)